In [1]:
import numpy as np
from copy import deepcopy

In [38]:
def fft(ingang, pattern=[0, 1, 0, -1], n=1, verbose=20):
    inlist = []
    
    if not type(ingang) == type(inlist):
        while ingang > 0:
            inlist.append(ingang % 10)
            ingang = ingang // 10
        inlist.reverse()
    else:
        inlist = ingang
    print('input', ''.join([str(e) for e in inlist]))
    
    for i in range(n):
#         if (i + 1) % verbose == 0:
#             print('cycle', i+1, 'i/p', ''.join([str(e) for e in inlist]))
        outlist = [0]*len(inlist)
        
        for j in range(len(inlist)):
            pat_i = []
            for pat_elem in pattern:
                pat_i += [pat_elem]*(j+1)
                
            while len(pat_i)-1 < len(inlist):
                pat_i += pat_i
            pat_i = pat_i[1:]
            
#             print('digit', j, 'pattern', pat_i)
            
            outlist[j] = sum([a*b for a, b in zip(inlist, pat_i)])
            outlist[j] = abs(outlist[j]) % 10
            
        inlist = outlist
        if (i + 1) % verbose == 0:
            print('cycle', i+1, 'o/p', ''.join([str(e) for e in outlist]))
            
    return outlist

In [39]:
fft(12345678, n=4)

input 12345678


[0, 1, 0, 2, 9, 4, 9, 8]

In [62]:
# 24176176
out = fft(80871224585914546619083218645595, n=100, verbose=1000)
print(''.join([str(e) for e in out[:8]]))

24176176


In [53]:
# 19617804207202209144916044189917 becomes 73745418
out = fft(19617804207202209144916044189917, n=100)
print(''.join([str(e) for e in out[:8]]))

cycle 20 i/p 47872553993941908195027629140847
cycle 20 o/p 76963953849540363545008153439917
cycle 40 i/p 05361729057504456438618145990847
cycle 40 o/p 33654966142448313730265762789917
cycle 60 i/p 11647603485383690629629161740847
cycle 60 o/p 72975119617377842264597871039917
cycle 80 i/p 96066219563743238763050687590847
cycle 80 o/p 95046448806853993582994480389917
cycle 100 i/p 93112646906349938755801603340847
cycle 100 o/p 73745418557257259149466599639917
73745418


In [54]:
# 69317163492948606335995924319873 becomes 52432133
out = fft(69317163492948606335995924319873, n=100)
print(''.join([str(e) for e in out[:8]]))

cycle 20 i/p 73122895087045524468924672471143
cycle 20 o/p 63274212574368112848019592069873
cycle 40 i/p 43201602449902737506732063621143
cycle 40 o/p 07019014885187866944818660719873
cycle 60 i/p 52214866813000119805470954871143
cycle 60 o/p 21118630151591935688392238469873
cycle 80 i/p 00150230627428127860048345021143
cycle 80 o/p 02811200895036156915551306119873
cycle 100 i/p 80986334163590178086546236271143
cycle 100 o/p 52432133292998606880495974869873
52432133


In [4]:
with open('calca_16_input.txt', 'r') as infile:
    instr = [row for row in infile][0]
    
inlist = [int(col) for col in instr if col.isdigit()]
out = fft(inlist, n=100, verbose=1000)
print(''.join([str(e) for e in out[:8]]))

27831665


In [117]:
k = [0]*400000
l = [1]*400000
m = [2]*400000

print(len(k), len(l), len(m))

400000 400000 400000


In [85]:
def makelist(instr):
    inlist = []
    if type(instr) == type(1):
        while instr > 0:
            inlist.append(instr%10)
            instr = instr // 10
        inlist.reverse()
    elif type(instr) == type('abc'):
        for d in instr:
            inlist.append(int(d))
    return inlist

def get_digits(inlist, inlength, level, i=0, N=8):
    digits = [0]*N
    for idx in range(i, i+N):
        dee = nphase(inlist, inlength, idx, level)
        digits[idx-i] = dee
    return digits

In [134]:
def digit_xy(inlist, inlen, offset, phase=1):
    if offset < inlen//2:
        print('get out')
        return 0
    
    phase_digit = 0
    for i in range(offset, inlen):
        if phase-1 == 0:
            idx = i%len(inlist)
            phase_digit += inlist[idx]
        else:
            phase_digit += digit_xy(inlist, inlen, i, phase-1)
    return phase_digit % 10

def digits_xy(inlist, inlen, offset, phase=1, N=8):
    outlist = [0]*N
    for i in range(N):
        digit = digit_xy(inlist, inlen, offset+i, phase)
        outlist[i] = digit
    print(''.join([str(e) for e in outlist]))
    return outlist

def super_digits(inlist, inlen, offset, phase=1, N=8):
    inlist_ex = [0]*(inlen-offset)
    for i in range(offset, inlen):
        idx = i % len(inlist)
        inlist_ex[i-offset] = inlist[idx]
        
    for p in range(phase):
        sumlist = [0]*(inlen-offset)
        i = len(inlist_ex) - 1
        sumlist[i] = inlist_ex[i]
        i -= 1
        
        while i >= 0:
            sumlist[i] = (sumlist[i+1] + inlist_ex[i]) % 10
            i -= 1
        inlist_ex = sumlist
        if (p+1) % 10 == 0:
            print('phase', p+1, sumlist[:N])
        
    return sumlist[:N]

In [125]:
instr = 12345678
inlist = makelist(instr)

super_digits(inlist, len(inlist), 5, 4, 3)

phase 1 [1, 5, 8]
phase 2 [4, 3, 8]
phase 3 [5, 1, 8]
phase 4 [4, 9, 8]


[4, 9, 8]

In [135]:
instr = '03036732577212944063491565474664'
inlist = makelist(instr)

ret = super_digits(inlist, len(inlist)*10000, 303673, phase=100, N=8)
print(''.join([str(r) for r in ret]))

phase 10 [4, 0, 7, 6, 2, 8, 2, 0]
phase 20 [9, 0, 9, 6, 8, 8, 2, 6]
phase 30 [2, 4, 7, 0, 0, 8, 2, 2]
phase 40 [6, 9, 2, 7, 4, 7, 7, 7]
phase 50 [3, 8, 0, 3, 0, 6, 4, 9]
phase 60 [0, 0, 2, 0, 7, 2, 1, 4]
phase 70 [2, 4, 4, 4, 4, 6, 4, 6]
phase 80 [8, 6, 8, 6, 0, 2, 0, 6]
phase 90 [2, 0, 4, 8, 6, 8, 2, 8]
phase 100 [8, 4, 4, 6, 2, 0, 2, 6]
84462026


In [138]:
# 02935109699940807407585447034323 becomes 78725270
instr = '02935109699940807407585447034323'
inlist = makelist(instr)

ret = super_digits(inlist, len(inlist)*10000, int(instr[:7]), phase=100, N=8)
print(''.join([str(r) for r in ret]))

phase 10 [4, 2, 0, 4, 8, 2, 1, 1]
phase 20 [4, 0, 8, 2, 6, 4, 6, 0]
phase 30 [2, 6, 2, 6, 8, 6, 0, 2]
phase 40 [6, 8, 6, 4, 0, 4, 2, 8]
phase 50 [2, 6, 6, 8, 6, 6, 2, 4]
phase 60 [8, 6, 8, 4, 0, 2, 0, 4]
phase 70 [0, 6, 4, 8, 8, 8, 6, 6]
phase 80 [8, 6, 8, 2, 0, 4, 4, 6]
phase 90 [6, 4, 6, 6, 4, 8, 8, 6]
phase 100 [7, 8, 7, 2, 5, 2, 7, 0]
78725270


In [139]:
# 03081770884921959731165446850517 becomes 53553731
instr = '03081770884921959731165446850517'
inlist = makelist(instr)

ret = super_digits(inlist, len(inlist)*10000, int(instr[:7]), phase=100, N=8)
print(''.join([str(r) for r in ret]))

phase 10 [8, 8, 5, 6, 6, 9, 4, 7]
phase 20 [4, 6, 4, 8, 9, 2, 8, 9]
phase 30 [2, 6, 6, 6, 4, 2, 0, 6]
phase 40 [8, 5, 7, 6, 3, 4, 8, 5]
phase 50 [3, 4, 5, 1, 0, 7, 9, 5]
phase 60 [9, 4, 5, 5, 4, 5, 5, 0]
phase 70 [8, 5, 1, 3, 1, 3, 3, 2]
phase 80 [7, 4, 0, 3, 3, 3, 2, 8]
phase 90 [4, 4, 4, 8, 8, 8, 8, 8]
phase 100 [5, 3, 5, 5, 3, 7, 3, 1]
53553731


In [143]:
with open('calca_16_input.txt', 'r') as infile:
    instr = [row.strip() for row in infile][0]

inlist = makelist(instr)

ret = super_digits(inlist, len(inlist)*10000, int(instr[:7]), phase=100, N=8)
print(''.join([str(r) for r in ret]))

phase 10 [7, 7, 1, 5, 3, 6, 9, 3]
phase 20 [8, 3, 2, 4, 2, 4, 5, 8]
phase 30 [9, 3, 8, 1, 9, 6, 1, 6]
phase 40 [2, 5, 6, 9, 2, 8, 7, 3]
phase 50 [7, 2, 8, 2, 4, 5, 7, 8]
phase 60 [2, 2, 4, 7, 9, 4, 6, 7]
phase 70 [7, 0, 2, 3, 2, 6, 5, 5]
phase 80 [8, 7, 0, 1, 3, 6, 2, 7]
phase 90 [1, 3, 7, 4, 1, 0, 9, 3]
phase 100 [3, 6, 2, 6, 5, 5, 8, 9]
36265589


In [23]:
def nphase(inlist, inlength, idx=0, level=1, pattern=[0, 1, 0, -1]):
    digit = 0
    patlength = len(pattern)*(idx+1)
    
    p = idx
    while p <= idx+idx:
        q = p
        while q < inlength:
            if level-1 == 0:
                a = inlist[q%len(inlist)]
            else:
                a = nphase(inlist, inlength, q, level-1)
            b = 0
            r = q + patlength//2
            if r < inlength:
                
                if level-1 == 0:
                    b = inlist[r%len(inlist)]
                else:
                    b = nphase(inlist, inlength, r, level-1)
            digit += a - b
            q += patlength
        p += 1
    return abs(digit)%10

In [74]:
with open('calca_16_input.txt', 'r') as infile:
    instr = [row for row in infile][0]
    
inlist = [int(col) for col in instr if col.isdigit()]

with open('output.txt', 'w') as outfile:
    idx = 0
    step = 1*len(inlist)
    lentotal = 1000*len(inlist)
    while idx < lentotal:
        step_i = step
        if idx + step > lentotal:
            step_i = lentotal - idx
        print('processing', idx, 'to', idx+step-1, 'max', lentotal-1)
        digits = get_digits(inlist, lentotal, 1, i=idx, N=step_i)
        outstr = ''.join([str(d) for d in digits])
        outfile.write(outstr)
        idx += step
    print('done')

processing 0 to 649 max 649999
processing 650 to 1299 max 649999
processing 1300 to 1949 max 649999
processing 1950 to 2599 max 649999
processing 2600 to 3249 max 649999
processing 3250 to 3899 max 649999
processing 3900 to 4549 max 649999
processing 4550 to 5199 max 649999
processing 5200 to 5849 max 649999
processing 5850 to 6499 max 649999
processing 6500 to 7149 max 649999
processing 7150 to 7799 max 649999
processing 7800 to 8449 max 649999
processing 8450 to 9099 max 649999
processing 9100 to 9749 max 649999
processing 9750 to 10399 max 649999
processing 10400 to 11049 max 649999
processing 11050 to 11699 max 649999
processing 11700 to 12349 max 649999
processing 12350 to 12999 max 649999
processing 13000 to 13649 max 649999
processing 13650 to 14299 max 649999
processing 14300 to 14949 max 649999
processing 14950 to 15599 max 649999
processing 15600 to 16249 max 649999
processing 16250 to 16899 max 649999
processing 16900 to 17549 max 649999
processing 17550 to 18199 max 649999


processing 142350 to 142999 max 649999
processing 143000 to 143649 max 649999
processing 143650 to 144299 max 649999
processing 144300 to 144949 max 649999
processing 144950 to 145599 max 649999
processing 145600 to 146249 max 649999
processing 146250 to 146899 max 649999
processing 146900 to 147549 max 649999
processing 147550 to 148199 max 649999
processing 148200 to 148849 max 649999
processing 148850 to 149499 max 649999
processing 149500 to 150149 max 649999
processing 150150 to 150799 max 649999
processing 150800 to 151449 max 649999
processing 151450 to 152099 max 649999
processing 152100 to 152749 max 649999
processing 152750 to 153399 max 649999
processing 153400 to 154049 max 649999
processing 154050 to 154699 max 649999
processing 154700 to 155349 max 649999
processing 155350 to 155999 max 649999
processing 156000 to 156649 max 649999
processing 156650 to 157299 max 649999
processing 157300 to 157949 max 649999
processing 157950 to 158599 max 649999
processing 158600 to 1592

processing 279500 to 280149 max 649999
processing 280150 to 280799 max 649999
processing 280800 to 281449 max 649999
processing 281450 to 282099 max 649999
processing 282100 to 282749 max 649999
processing 282750 to 283399 max 649999
processing 283400 to 284049 max 649999
processing 284050 to 284699 max 649999
processing 284700 to 285349 max 649999
processing 285350 to 285999 max 649999
processing 286000 to 286649 max 649999
processing 286650 to 287299 max 649999
processing 287300 to 287949 max 649999
processing 287950 to 288599 max 649999
processing 288600 to 289249 max 649999
processing 289250 to 289899 max 649999
processing 289900 to 290549 max 649999
processing 290550 to 291199 max 649999
processing 291200 to 291849 max 649999
processing 291850 to 292499 max 649999
processing 292500 to 293149 max 649999
processing 293150 to 293799 max 649999
processing 293800 to 294449 max 649999
processing 294450 to 295099 max 649999
processing 295100 to 295749 max 649999
processing 295750 to 2963

processing 416650 to 417299 max 649999
processing 417300 to 417949 max 649999
processing 417950 to 418599 max 649999
processing 418600 to 419249 max 649999
processing 419250 to 419899 max 649999
processing 419900 to 420549 max 649999
processing 420550 to 421199 max 649999
processing 421200 to 421849 max 649999
processing 421850 to 422499 max 649999
processing 422500 to 423149 max 649999
processing 423150 to 423799 max 649999
processing 423800 to 424449 max 649999
processing 424450 to 425099 max 649999
processing 425100 to 425749 max 649999
processing 425750 to 426399 max 649999
processing 426400 to 427049 max 649999
processing 427050 to 427699 max 649999
processing 427700 to 428349 max 649999
processing 428350 to 428999 max 649999
processing 429000 to 429649 max 649999
processing 429650 to 430299 max 649999
processing 430300 to 430949 max 649999
processing 430950 to 431599 max 649999
processing 431600 to 432249 max 649999
processing 432250 to 432899 max 649999
processing 432900 to 4335

processing 553800 to 554449 max 649999
processing 554450 to 555099 max 649999
processing 555100 to 555749 max 649999
processing 555750 to 556399 max 649999
processing 556400 to 557049 max 649999
processing 557050 to 557699 max 649999
processing 557700 to 558349 max 649999
processing 558350 to 558999 max 649999
processing 559000 to 559649 max 649999
processing 559650 to 560299 max 649999
processing 560300 to 560949 max 649999
processing 560950 to 561599 max 649999
processing 561600 to 562249 max 649999
processing 562250 to 562899 max 649999
processing 562900 to 563549 max 649999
processing 563550 to 564199 max 649999
processing 564200 to 564849 max 649999
processing 564850 to 565499 max 649999
processing 565500 to 566149 max 649999
processing 566150 to 566799 max 649999
processing 566800 to 567449 max 649999
processing 567450 to 568099 max 649999
processing 568100 to 568749 max 649999
processing 568750 to 569399 max 649999
processing 569400 to 570049 max 649999
processing 570050 to 5706

In [29]:
instr = 12345678
inlist = makelist(instr)

digits = get_digits(inlist, len(inlist), 4, i=0, N=5)
print(digits)
    
print(outgits)
# After 4 phases: 01029498

[0, 1, 0, 2, 9]
[0, 1, 0, 2, 9, 4, 9, 8]


In [62]:
#     80871224585914546619083218645595 becomes 24176176.
#     19617804207202209144916044189917 becomes 73745418.
#     69317163492948606335995924319873 becomes 52432133.

instr = 80871224585914546619083218645595
inlist = makelist(instr)
digits = get_digits(inlist, len(inlist), 1, i=0, N=len(inlist))
print(''.join([str(d) for d in digits]), 'from', instr)

instr = 19617804207202209144916044189917
inlist = makelist(instr)
digits = get_digits(inlist, len(inlist), 1, i=0, N=len(inlist))
print(''.join([str(d) for d in digits]), 'from', instr)

instr = 69317163492948606335995924319873
inlist = makelist(instr)
digits = get_digits(inlist, len(inlist), 1, i=0, N=len(inlist))
print(''.join([str(d) for d in digits]), 'from', instr)

24706861300603878265668532484945 from 80871224585914546619083218645595
43831307100027207873909339546787 from 19617804207202209144916044189917
24292942725792436074901675187803 from 69317163492948606335995924319873


In [41]:
out = fft(instr, n=100, verbose=1)

input 80871224585914546619083218645595
cycle 1 o/p 24706861300603878265668532484945
cycle 2 o/p 17934388121573275759482496402895
cycle 3 o/p 72859053525425497250179734844245
cycle 4 o/p 00978736644511462538870141795195
cycle 5 o/p 51836416307007065385792217690545
cycle 6 o/p 98521171128258098302701976934495
cycle 7 o/p 31240534972599817966477670452845
cycle 8 o/p 50348847966861937015958158849795
cycle 9 o/p 09220321365200291443894650240145
cycle 10 o/p 87380253238657140951801716640095
cycle 11 o/p 42470332244796312238799810484445
cycle 12 o/p 58216774982683948641367809957395
cycle 13 o/p 33622721914577690262182577894745
cycle 14 o/p 56133991119520617759768614790695
cycle 15 o/p 83578817438354326927815710690045
cycle 16 o/p 60772830467501670453687254489995
cycle 17 o/p 64235811884242808849602538402345
cycle 18 o/p 34492308665397761351266496844295
cycle 19 o/p 81468145869661265416537178240645
cycle 20 o/p 47119211811248198398274769195595
cycle 21 o/p 95191085875562817967970693434945
cycl